# Prepare doctor data for use in simulation
* save adjacency number, num of patients, capacity and gemeinde info

In [1]:
import pandas as pd
from os.path import join

## Set criteria for patients, timeframe and capacity

In [2]:
# information selection criteria
patient_type = 'total'
capacity_type = 'hour-based'
timeframe = 'quarterly'
threshold = 0.9             # selected capacity threshold

## Load and transform data

In [3]:
data_path = 'data'
docs = pd.read_csv(join(data_path, 'matched_and_imputed_doctors_with_capacity_threshold{}.csv'.format(threshold)))

In [4]:
# add district-column (first 3 digits of municipality number)
docs['bezirk'] = docs['gemeinde'].apply(lambda x: str(x)[0:3])
docs['bezirk'] = docs['bezirk'].astype(int)

In [5]:
# add federal state-column (first digits of district number)
docs['bundesland'] = docs['gemeinde'].apply(lambda x: str(x)[0])
docs['bundesland'] = docs['bundesland'].astype(int)

In [6]:
doctors = docs.specialization.unique()

### Add names of all federal states

In [7]:
### fill in names of districs and states
docs.loc[docs.bundesland==3,'bundesland'] = 'Niederösterreich'
docs.loc[docs.bundesland==1,'bundesland'] = 'Burgenland'
docs.loc[docs.bundesland==2,'bundesland'] = 'Kärnten'
docs.loc[docs.bundesland==4,'bundesland'] = 'Oberösterreich'
docs.loc[docs.bundesland==5,'bundesland'] = 'Salzburg_BL'
docs.loc[docs.bundesland==6,'bundesland'] = 'Steiermark'
docs.loc[docs.bundesland==7,'bundesland'] = 'Tirol'
docs.loc[docs.bundesland==8,'bundesland'] = 'Voralberg'
docs.loc[docs.bundesland==9,'bundesland'] = 'Wien'

docs.loc[docs.bezirk==101,'bezirk'] = 'Eisenstadt'
docs.loc[docs.bezirk==201,'bezirk'] = 'Klagenfurt'
docs.loc[docs.bezirk==302,'bezirk'] = 'StPolten'
docs.loc[docs.bezirk==401,'bezirk'] = 'Linz'
docs.loc[docs.bezirk==501,'bezirk'] = 'Salzburg'
docs.loc[docs.bezirk==601,'bezirk'] = 'Graz'
docs.loc[docs.bezirk==701,'bezirk'] = 'Innsbruck'
docs.loc[docs.bezirk==802,'bezirk'] = 'Bregenz'

In [8]:
### use whole country now as underlying network
bezirk = 'Österreich'

In [9]:
docs.bezirk = docs.bezirk.astype(str)

## Loop over districts and doctors

In [10]:
for specialization in doctors:
    # for all austria
    doc_save = docs[docs['specialization'] == specialization]

    ## Select information (capacity, # patients)
    N_patients_col = 'N_{}_{}_patients'.format(patient_type, timeframe)
    capacity_col = 'capacity_{}_{}'.format(patient_type, capacity_type)

    doc_save = doc_save[['adj_index', N_patients_col, capacity_col,'gemeinde']]  # added 'gemeinde' 22.12.20
    doc_save = doc_save.rename(columns={N_patients_col:'number_of_patients',
                        capacity_col:'capacity'})

    ## Save data
    doc_save.reset_index(drop=True, inplace=True)
    doc_save.to_csv(join(data_path, 'doctor_info_bez={}_spec={}_ptype={}_ctype={}_tf={}_th={}.csv'\
            .format(bezirk, specialization, patient_type, capacity_type, timeframe, threshold)), index=False)